# Word Embeddings : le modèle Word2Vec

## Imports

In [20]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [21]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [22]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [23]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [24]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [25]:
len(bigram_phrases.vocab.keys())

15850647

Prenons une clé au hasard :

In [26]:
key_= list(bigram_phrases.vocab.keys())[144]
print(key_)

1q


Le dictionnaire indique le score de cette coocurrence :

In [27]:
bigram_phrases.vocab[key_]

488

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [28]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [29]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [30]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [31]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [32]:
print(corpus[:100])

[['mi', 'imnri', 'r', 'i', '<<', 'i', 'i', 'hmu', "'", 'i', '/', 'tx', "-'", 'l', ':', 'marche', 'tenu', 'hors', 'villa', ',', 'la', '9', '.'], ['--', 'u', 'a', 'ete', 'vaain', 'si', 'teicj', '>>', 'm', 'races_indigenes', 'de', 'fr', '.'], ['31', '<)', 'a', '5s', "'", 'k', '131', 'de', '.'], ['rasa', 'iichakdui', "'", 'te', ',', 'do', '(', 'r', '.', '3s0', 'h', '710', '.', 'taureaux', 'iallsenas', ',>', 'ia', 'u', '\\', '--', 'a', '--', ';', '0ii', '.'], ['hollandais', ',', 'dufr', '.'], ['0', '.'], ['--', 'a', '9', '.--', 'la', 'idto', '-', 'vachei', 'laitieres', ':', 'bn', 'vante', '1q', '.'], ['vendues', '3', '\\', 'au', 'prix', 'la', '410', 'a', '*', '<<', 'i', 'h', '\\;', 'genisses', ',', 'kl', '.'], ["'.", '9', '.'], ['i', 'l', '.', '2', 'i', '.', 'id', '.'], ['da', '370', 'i', '6lutr', '.'], ['marche', 'a', '<', 'u', 'porcs', '.'], ['--', 'categorie', 'de', 'lt', 'ilashtya', ':', "'", '237', 'on', 'vente', ';', 'vendus', '1', 'm', '.', 'do', "'", '2', 'i', '.--', 'a', ';:,', 'l'

## Entrainement d'un modèle Word2Vec sur ce corpus

In [52]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=6, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=3, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=7 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 5min 28s
Wall time: 9min 35s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [53]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [54]:
model = Word2Vec.load("../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [61]:
model.wv["dieu"]

array([ 1.6888598 , -1.0672638 , -2.2422738 , -1.9001126 , -2.4432218 ,
       -1.46235   , -0.9715014 , -1.3654633 , -3.6785505 ,  0.29820517,
        1.8520144 , -3.835808  , -4.2769694 , -5.91797   , -1.3427918 ,
        2.7583995 ,  0.5818314 ,  0.85508615, -0.36045277, -0.59332067,
       -4.428832  , -0.6219408 ,  0.4549607 , -1.9093367 ,  3.5782638 ,
       -1.3537786 ,  4.280129  , -6.597984  ,  3.673077  , -2.3137038 ,
       -4.8271008 , -2.9477067 ], dtype=float32)

### Calculer la similarité entre deux termes

In [65]:
model.wv.similarity("dieu", "diable")

0.80468637

### Chercher les mots les plus proches d'un terme donné

In [68]:
model.wv.most_similar("dieu", topn=10)

[('chagrin', 0.8588914275169373),
 ('papa', 0.8446003794670105),
 ('monseigneur', 0.8379319906234741),
 ('parrain', 0.8274325728416443),
 ('cet_enfant', 0.8255584239959717),
 ('mon_pere', 0.8232569694519043),
 ('cet_homme', 0.8208887577056885),
 ('ma_mere', 0.8206090927124023),
 ('seigneur', 0.8187078833580017),
 ('napoleon', 0.8048292398452759)]

### Faire des recherches complexes à travers l'espace vectoriel

In [69]:
print(model.wv.most_similar(positive=['dieu', 'jesus'], negative=['chagrin']))

[('cleopatre', 0.7815219163894653), ('corneille', 0.7611695528030396), ('ronsard', 0.7574043273925781), ('themis', 0.7563536763191223), ('salomon', 0.7497104406356812), ('napoleon', 0.7363564372062683), ('goethe', 0.7356658577919006), ('jean_anouilh', 0.7343509197235107), ('castille', 0.7338094711303711), ('moise', 0.732423722743988)]
